**NAME: PARTHA SAKHA PAUL**

**ROLL: MA23M016**

    CS6910_assignment 2


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls "/content/drive/My Drive"

'Colab Notebooks'	   ma23m016_lab7.mp4	     ma23m016_video_lab3.mp4
'Copy of template.gdoc'    ma23m016_lab8.mp4	     ma23m016_video_lab4.mp4
 ma23m016_lab10.mp4	   ma23m016_lab9.mp4	     nature_12K
 ma23m016_lab5_video.mp4   ma23m016-video-hw1.webm   nature_12K.zip
 ma23m016_lab6.mp4	   ma23m016_video_hw2.mp4    Screenshot_20240207-220942_GPay.jpg


In [ ]:
!unzip '/content/drive/My Drive/nature_12K.zip' -d '/content/drive/My Drive/nature_12K'

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms

data_dir = '/content/drive/My Drive/nature_12K'

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

dataset = datasets.ImageFolder(root=data_dir, transform=transform)

In [3]:
from torchvision import datasets, transforms
from torch.utils.data.sampler import SubsetRandomSampler
import numpy as np

data_dir = '/content/drive/My Drive/nature_12K/inaturalist_12K'

# Defining transforms for the training, validation, and testing sets
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Load the datasets with ImageFolder
train_data = datasets.ImageFolder(data_dir + '/train', transform=transform)
print(len(train_data))
# print(train_data[9998])
# Creating data indices for training and validation splits:
dataset_size = len(train_data)  #9999
indices = list(range(dataset_size)) #[0,1,...,9998]
split = int(np.floor(0.2 * dataset_size))  #1999
print(split)
np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]  # [1999,...,9998] , [0,...,1998]
# print((val_indices))

# Creating data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)
# print(type(train_sampler))

train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, sampler=train_sampler)
# print(len(train_loader))
validation_loader = torch.utils.data.DataLoader(train_data, batch_size=64, sampler=valid_sampler)
# print(*validation_loader)

9999
1999


In [12]:
# defining a class my_CNN that inherits from nn.Module which is PyTorch's base class for all neural network
class my_CNN(nn.Module):
    def __init__(self, num_classes=10, num_filters=[32, 64, 128, 256, 512], filter_sizes=[3, 3, 3, 3, 3], activation_fn=F.relu, num_neurons_dense=512):
        super(my_CNN, self).__init__()
        # Initializing class variables
        self.num_classes = num_classes
        self.num_filters = num_filters
        self.filter_sizes = filter_sizes
        self.activation_fn = activation_fn
        self.num_neurons_dense = num_neurons_dense

        # Creating convolution layers using ModuleList
        self.conv_layers = nn.ModuleList()
        in_channels = 3  # since, input images are RGB
        for out_channels, kernel_size in zip(num_filters, filter_sizes):
            # a convolutional layer with in-out channels and kernel size
            conv_layer = nn.Conv2d(in_channels, out_channels, kernel_size, padding=0)
            self.conv_layers.append(conv_layer)
            # Update in_channels for the next layer
            in_channels = out_channels

        # Placeholder for dense layer will be initialized after calculating the flatten size
        self.dense = None
        # output layer: maps from the dense layer to the number of classes.
        self.out = nn.Linear(num_neurons_dense, num_classes)
        # find the size needed to flatten the conv layer outputs, initializing the dense layer accordingly
        self.init_flatten_size(input_shape=(3, 224, 224))  # input images are 224x224 RGB images
        # initializing the dense layer from flatten size
        self.dense = nn.Linear(self.flatten_size, num_neurons_dense)


    def init_flatten_size(self, input_shape):
        # to disable gradient calculations for speed up this computation
        with torch.no_grad():
            # a dummy input tensor of the correct shape
            input_tensor = torch.zeros(1, *input_shape)
            # Forward propagate through the conv layers to find output size
            output = self.forward_conv_layers(input_tensor)
            # total number of output features is the flat size needed for the dense layer input
            self.flatten_size = output.numel()

    # a function for sequentially passing input through all convolutional layers and applying activation and pooling
    def forward_conv_layers(self, x):
        for conv in self.conv_layers:
            x = conv(x)
            x = self.activation_fn(x)
            x = F.max_pool2d(x, 2)
        return x

    def forward(self, x):
        # Convolution blocks
        for conv in self.conv_layers:
            x = conv(x)
            x = self.activation_fn(x)
            x = F.max_pool2d(x, 2)

        # Flattening the output for the dense layer
        x = torch.flatten(x, 1)
        # Dense layer
        x = self.dense(x)
        x = self.activation_fn(x)
        # Output layer
        x = self.out(x)
        return x

# model with parameters
model = my_CNN(num_classes=10,
                  num_filters=[32, 64, 128, 256, 512],
                  filter_sizes=[3, 3, 3, 3, 3],
                  activation_fn=F.relu,
                  num_neurons_dense=512)

print(model)

my_CNN(
  (conv_layers): ModuleList(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
    (1): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
    (3): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1))
    (4): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1))
  )
  (out): Linear(in_features=512, out_features=10, bias=True)
  (dense): Linear(in_features=12800, out_features=512, bias=True)
)
